In [1]:
import pickle
import sklearn
import modAL
from modAL.models import ActiveLearner
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer
from sklearn.model_selection import GridSearchCV
import numpy as np
from modAL.uncertainty import uncertainty_sampling
from modAL.uncertainty import entropy_sampling
from modAL.uncertainty import margin_sampling
import sys
import os

import pandas as pd
import csv

# import audio_decryption
import librosa
import IPython.display
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path_project = '/beegfs/amm1209/ismir2018_LBD/'

# 1. Load training data

In [20]:
X_train = pickle.load(open(os.path.join(path_project + 'X_train.pickle'), "rb" ))
y_train = pickle.load(open(os.path.join(path_project + 'y_train.pickle'), "rb" ))
X_val = pickle.load(open(os.path.join(path_project + 'X_val.pickle'), "rb" ))
y_val = pickle.load(open(os.path.join(path_project + 'y_val.pickle'), "rb" ))
X_test = pickle.load(open(os.path.join(path_project + 'X_test.pickle'), "rb" ))
y_test = pickle.load(open(os.path.join(path_project + 'y_test.pickle'), "rb" ))

X_pool = pickle.load(open(os.path.join(path_project + 'X_pool.pickle'), "rb" ))
key_pool = pickle.load(open(os.path.join(path_project + 'key_pool.pickle'), "rb" ))

In [21]:
print(X_train)

[[ 199.           51.3         234.2        ...,   41.21516711
    30.98725544    0.        ]
 [ 179.2          37.3         177.1        ...,   82.39271812
    23.15620867    0.        ]
 [ 191.1          39.          197.4        ...,   78.67934926
    49.31368978    0.        ]
 ..., 
 [   0.            0.            0.         ...,    0.            0.            0.        ]
 [ 185.1          19.2         176.3        ...,    0.            0.            0.        ]
 [   0.            0.            0.         ...,    0.            0.            0.        ]]


In [22]:
print(X_train.shape)

(377, 256)


In [23]:
X_train = X_train[0:10,:]
#X_train = np.reshape(X_train,(1,-1))
print(X_train.shape)

(10, 256)


In [24]:
y_train = y_train[0:10]

In [25]:
y_train.shape

(10,)

# 2. Initial classifier

In [26]:
clf = RandomForestClassifier(n_estimators = 1000, max_depth=8, random_state=0)
clf.fit(X_train, y_train)
score = np.mean(clf.predict(X_test) == y_test)
print('accuracy:', score)

accuracy: 0.612587412587


In [27]:
pred = clf.predict(X_test)
starting_precision = precision_score(y_test,pred)
starting_recall = recall_score(y_test,pred)
starting_confusion_matrix = confusion_matrix(y_test,pred)
print(starting_precision)
print(starting_recall)
print(starting_confusion_matrix)

0.61095505618
1.0
[[  3 277]
 [  0 435]]


In [28]:
with open(os.path.join(path_project, 'models', 'model_initial_voice_n1000_d8.pickle'), 'wb') as f:
    pickle.dump(clf, f, protocol=pickle.HIGHEST_PROTOCOL)

# 3. Active Learning

In [29]:
learner = ActiveLearner(
    estimator=clf,
    query_strategy=modAL.uncertainty.uncertainty_sampling,
    X_training=X_train, y_training=y_train
)

In [30]:
def adjusted_classes(y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    Will only work for binary classification problems.
    """
    return [1 if y >= t else 0 for y in y_scores]

In [31]:
def active_learning(X_pool, key_pool, n_queries, n_instances, threshold, converge_cond, string):
    precision = [starting_precision]
    recall = [starting_recall]
    confusion_matrices = [starting_confusion_matrix]
    count = 1
    queries = {}
    not_converge = True
    
    while count < n_queries+1 and not_converge:
        y = []
        query_idx, query_instance = learner.query(X_pool, n_instances=n_instances)
        for i in range(n_instances):
            key = key_pool[query_idx[i]][0]
            filepath = os.path.join(path_project, 'fma-audio', str(key) +'.ogg')
            
            y_track, sr = librosa.core.load(filepath)
            IPython.display.display(IPython.display.Audio(data=y_track, rate=sr))

            y_new = input("Please input label for\nsample_key = '%s' (1 if voice is present, 0 if voice is not present)" % (key))
            y.append(int(y_new))
            
            queries[key] = int(y_new)
            
        print('ARRAY: ',y)
        learner.teach(
            X=X_pool[query_idx],
            y=np.array(y).reshape(-1, ))
    
        X_pool = np.delete(X_pool, query_idx, axis=0)
        key_pool = np.delete(key_pool, query_idx, axis=0)
        
        prediction_probabilities = learner.predict_proba(X_val)
        #if prediction_probabilities.shape[1]==1:
            #print("in if")
            #zeros = np.zeros((prediction_probabilities.shape[0],1))
            #prediction_probabilities = np.concatenate((prediction_probabilities,zeros), axis = 1)
        print(prediction_probabilities.shape)
        
        prediction = adjusted_classes(prediction_probabilities[:,1], threshold)
        
        precision.append(precision_score(y_val,prediction))
        recall.append(recall_score(y_val,prediction))
        confusion_matrices.append(confusion_matrix(y_val,prediction))
    
        print('Precision after query no. %d: %f' % (count, precision_score(y_val,prediction)))      
        print(precision[count]-precision[count-1])
        print('Recall after query no. %d: %f' % (count, recall_score(y_val,prediction))) 
        
        if len(precision) > 4 and np.abs(precision[count]-precision[count-1]) < converge_cond \
        and np.abs(precision[count-1]-precision[count-2]) < converge_cond \
        and np.abs(precision[count-2]-precision[count-3]) < converge_cond \
        and np.abs(recall[count]-recall[count-1]) < converge_cond \
        and np.abs(recall[count-1]-recall[count-2]) < converge_cond \
        and np.abs(recall[count-2]-recall[count-3]) < converge_cond:
            not_converge = False
        
        with open(os.path.join(path_project, 'models', string + str(count)+'.pickle'), 'wb') as f:
            pickle.dump(learner, f, protocol=pickle.HIGHEST_PROTOCOL)
        count += 1
    
    return queries, precision, recall, confusion_matrices

In [17]:
#active_learning(X_pool, key_pool, 100, 1, 0.5, 10e-4, 'voice_model_10_n10_d8'+'_iter_')

In [18]:
#active_learning(X_pool, key_pool, 100, 1, 0.5, 10e-4, 'voice_model_100_n100_d8'+'_iter_')

In [33]:
#active_learning(X_pool, key_pool, 100, 1, 0.5, 10e-4, 'voice_model_100_n1000_d8'+'_iter_')